# make a new function to make a 2D slice based on 1D Earth model (or 2D or 3D)

In [1]:
# Nobuaki Fuji Oct 2025

# breaking news! confirmed by Lorette and Stéphanie, the CMB is ellipsoidal! 
# when working with geodetic earth models, r_PREM should be corrected as a function of latitude!

using Pkg


cd(@__DIR__)
Pkg.activate("../..")
ParamFile = "../test/testparam.csv"
include("../src/batchRevise.jl") 

myInclude("../src/DSM1D.jl")
using .DSM1D

  Activating project at `~/Documents/Github/flexibleDSM`
┌ Info: Including with Revise: ../src/DSM1D.jl
└ @ Main /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/src/batchRevise.jl:25


  0.004139 seconds (2.17 k allocations: 108.156 KiB, 90.99% compilation time)


In [2]:
using Geodesy, Interpolations, StaticArrays, GMT, LinearAlgebra

In [19]:
DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer

6371.0

In [20]:
# Example 1-D model (toy). Replace with your actual model arrays.
# r_model in meters from Earth center (0..R_ref), assume sorted ascending
R_ref = DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer*1.e3                   # model reference radius (m)
r_model = collect(0.0:1000.0:R_ref)  # example radius grid
v_model = 5000 .- 0.5 .* (R_ref .- r_model) ./ 1000  # dummy profile

6372-element Vector{Float64}:
 1814.5
 1815.0
 1815.5
 1816.0
 1816.5
 1817.0
 1817.5
 1818.0
 1818.5
 1819.0
    ⋮
 4996.0
 4996.5
 4997.0
 4997.5
 4998.0
 4998.5
 4999.0
 4999.5
 5000.0

In [5]:
# create an interpolator in radius space (ensure interpolation domain covers used r)
itp = LinearInterpolation(r_model, v_model; extrapolation_bc=Flat())

6372-element extrapolate(interpolate((::Vector{Float64},), ::Vector{Float64}, Gridded(Linear())), Flat()) with element type Float64:
 1814.5
 1815.0
 1815.5
 1816.0
 1816.5
 1817.0
 1817.5
 1818.0
 1818.5
 1819.0
    ⋮
 4996.0
 4996.5
 4997.0
 4997.5
 4998.0
 4998.5
 4999.0
 4999.5
 5000.0

In [118]:
import Base: +,-,/
struct GeoPoint
    lat::Float64 # in degree
    lon::Float64 # in degree
    alt::Float64 # in metre
    ecef::SVector{3,Float64}
    radius::Float64 # in metre
    #effectiveRadius::Float64 # in metre (useful to get values from 1D averaged model)
end

function GeoPoint(lat::Float64, lon::Float64; alt=0.0, ell=wgs84)
    lla = LLA(lat,lon, alt) # be careful LLA uses degrees by default!!
    ecef_coords = ECEF(lla,ell)
    radius = norm([ecef_coords.x,ecef_coords.y,ecef_coords.z])
    GeoPoint(lat, lon, alt, SVector(ecef_coords.x, ecef_coords.y, ecef_coords.z),radius)
end


function GeoPoint(ecef::SVector{3,Float64}; ell=wgs84)
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function +(a::GeoPoint,b::GeoPoint; ell=wgs84)
    ecef=a.ecef + b.ecef
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function -(a::GeoPoint,b::GeoPoint; ell=wgs84)
    ecef=a.ecef - b.ecef
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end

function /(a::GeoPoint,c::Real; ell=wgs84)
    ecef=a.ecef / c
    lla = LLA(ECEF(ecef...),ell)
    radius = norm(ecef)
    GeoPoint(lla.lat,lla.lon,lla.alt,ecef,radius)
end


function effectiveRadius(a::GeoPoint,r0::Float64; ell=wgs84)
    radiusPlanetHere = GeoPoint(a.lat,a.lon).radius 
    ratio = r0/radiusPlanetHere
    return a.radius*ratio
end

function topographyFromGeoPoint(p::GeoPoint; dlon=0.01, dlat=0.01, precision="@earth_relief_01m")
    region = [p.lon - dlon, p.lon + dlon, p.lat - dlat, p.lat + dlat]
    topo = GMT.grdcut(precision, region=region)
    return GMT.grd2xyz(topo).z[1]  # or mean(topo.z) if you prefer
end


topographyFromGeoPoint (generic function with 1 method)

In [83]:
p1 = GeoPoint(48.8566,2.3522) # Paris
p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)
#topographyFromGeoPoint(p1)
@show GeoPoint(p1.ecef)
@show p0 = (p1 + p2)/2.0


GeoPoint(p1.ecef) = GeoPoint(48.8566, 2.3522000000000003, 7.047140324999535e-10, [4.200914795649138e6, 172559.77631070156, 4.780081341967356e6], 6.366053693814456e6)
p0 = (p1 + p2) / 2.0 = GeoPoint(45.82989800475024, 1.9029714540760907, -8978.352692728113, [4.443193158578102e6, 147626.58304377226, 4.545653677986095e6], 6.358201507709939e6)


GeoPoint(45.82989800475024, 1.9029714540760907, -8978.352692728113, [4.443193158578102e6, 147626.58304377226, 4.545653677986095e6], 6.358201507709939e6)

In [84]:
# pre-definition of x-axis vector p1->p2 (normally this is not the one we want)
p2_1 = p2-p1
x_axis_tentative = (p2_1/p2_1.radius).ecef


3-element SVector{3, Float64} with indices SOneTo(3):
  0.7166970175507206
 -0.07375625720924696
 -0.693473431038555

In [85]:
# definition of z-axis vector p0 -> p0_surface

z_axis = (p0/p0.radius).ecef

3-element SVector{3, Float64} with indices SOneTo(3):
 0.698812888076966
 0.023218292604404035
 0.7149275895823759

In [86]:
# y-axis: complete right-handed system
y_axis = normalize(cross(z_axis, x_axis_tentative))

3-element SVector{3, Float64} with indices SOneTo(3):
  0.036629318622305354
  0.9970002078796852
 -0.06818268478968828

In [87]:
x_axis = cross(y_axis, z_axis)  # now perfectly orthogonal

3-element SVector{3, Float64} with indices SOneTo(3):
  0.7143660409585519
 -0.07383424934541309
 -0.6958661244440657

In [88]:
#Rotation matrix
R = SMatrix{3,3,Float64}(
    x_axis[1], y_axis[1], z_axis[1],
    x_axis[2], y_axis[2], z_axis[2],
    x_axis[3], y_axis[3], z_axis[3]
)

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 0.714366   -0.0738342  -0.695866
 0.0366293   0.997      -0.0681827
 0.698813    0.0232183   0.714928

In [89]:
p_2D_to_ECEF(x_2D,z_2D,pOrigin::SVector{3,Float64},R::SMatrix{3,3,Float64}) = pOrigin+R*SVector(x_2D,0.e0,z_2D)

p_2D_to_ECEF (generic function with 1 method)

In [90]:
p_ECEF_to_2D(p_3D::SVector{3,Float64},pOrigin::SVector{3,Float64},R::SMatrix{3,3,Float64}) = R' * (p_3D - pOrigin)

p_ECEF_to_2D (generic function with 1 method)

In [91]:
p_ECEF_to_2D(p0.ecef, p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
    8341.076143154814
  -48189.85002661229
 -334492.0981322616

In [96]:
p0_1m_above=p_2D_to_ECEF(0,1,p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
      4.200914099783014e6
 172559.70812801676
      4.780082056894946e6

In [97]:
p_ECEF_to_2D(p0_1m_above, p1.ecef,R)

3-element SVector{3, Float64} with indices SOneTo(3):
 -2.2947774760174783e-10
  6.235097495158809e-12
  1.0000000001379907

In [99]:
Δx = 500 # in metre
Δz = 500

altMax = 5000 # in metre
altMin = -400.e3 # in metre

-400000.0

In [100]:
# since p1 and p2 can be even out of the solid Earth or inside the Earth
# I define that p1 and p2 should be the left and right limits

In [101]:
leftLimit = 0 # in metre so x is always non-negative
rightLimit = p2_1.radius


676097.0312306838

In [109]:
@show Nx = Int64((rightLimit-leftLimit) ÷ Δx+1) 
@show Nz = Int64((altMax-altMin) ÷ Δz + 1 ) 

Nx = Int64((rightLimit - leftLimit) ÷ Δx + 1) = 1353
Nz = Int64((altMax - altMin) ÷ Δz + 1) = 811


811

In [125]:
allGridsInGeoPoints=Array{GeoPoint,2}(undef,Nx,Nz)
effectiveRadii=Array{Float64,2}(undef,Nx,Nz)
for iXZ in CartesianIndices(allGridsInGeoPoints)
    ix, iz = Tuple(iXZ)
    x = leftLimit+(ix-1)*Δx
    z = altMin+(iz-1)*Δz 
    tmpGeoPoint=GeoPoint(p_2D_to_ECEF(x,z,p1.ecef,R))
    allGridsInGeoPoints[iXZ]=tmpGeoPoint
    effectiveRadii[iXZ]=effectiveRadius(tmpGeoPoint,DSM1D.my1DDSMmodel.averagedPlanetRadiusInKilometer*1.e3 )
end

In [126]:
allGridsInGeoPoints[3,3]

GeoPoint(45.26820230196961, 2.5540239364680706, -18090.050762195096, [4.479279745343279e6, 199801.2968604095, 4.495524046612065e6], 6.349299492773748e6)

In [127]:
effectiveRadii[200,1]

6.4514247701887805e6

In [128]:
minimum(effectiveRadii)

6.351882957945972e6